In [107]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

In [108]:
url = "https://keithgalli.github.io/web-scraping/"

#Load webpage content
r = requests.get( url + "webpage.html")

# Convert to t a beautifyl soup object
webpage = bs(r.content)

# Render out the html  
print(webpage)

#To render in better format
print(webpage.prettify())

<html><head>
<title>Keith Galli's Page</title>
<style>
  table {
    border-collapse: collapse;
  }
  th {
    padding:5px;
  }
  td {
    border: 1px solid #ddd;
    padding: 5px;
  }
  tr:nth-child(even) {
    background-color: #f2f2f2;
  }
  th {
    padding-top: 12px;
    padding-bottom: 12px;
    text-align: left;
    background-color: #add8e6;
    color: black;
  }
  .block {
  width: 100px;
  /*float: left;*/
    display: inline-block;
    zoom: 1;
  }
  .column {
  float: left;
  height: 200px;
  /*width: 33.33%;*/
  padding: 5px;
  }

  .row::after {
    content: "";
    clear: both;
    display: table;
  }
</style>
</head>
<body>
<h1>Welcome to my page!</h1>
<img src="./images/selfie1.jpg" width="300px"/>
<h2>About me</h2>
<p>Hi, my name is Keith and I am a YouTuber who focuses on content related to programming, data science, and machine learning!</p>
<p>Here is a link to my channel: <a href="https://www.youtube.com/kgmit">youtube.com/kgmit</a></p>
<p>I grew up in the great s

## i. Grab all social links from the webpage

### Method 1

In [109]:
links = webpage.select("ul.socials a")
actual_links = [link["href"] for link in links]
actual_links

['https://www.instagram.com/keithgalli/',
 'https://twitter.com/keithgalli',
 'https://www.linkedin.com/in/keithgalli/',
 'https://www.tiktok.com/@keithgalli']

### Method 2

In [110]:
ulist = webpage.find("ul" , attrs = {"class" : "socials"})
links = ulist.find_all("a")
actual_links = [link["href"] for link in links]
actual_links

['https://www.instagram.com/keithgalli/',
 'https://twitter.com/keithgalli',
 'https://www.linkedin.com/in/keithgalli/',
 'https://www.tiktok.com/@keithgalli']

### Method 3

In [111]:
links = webpage.select("li.social a")
actual_links = [link["href"] for link in links]
actual_links

['https://www.instagram.com/keithgalli/',
 'https://twitter.com/keithgalli',
 'https://www.linkedin.com/in/keithgalli/',
 'https://www.tiktok.com/@keithgalli']

## ii. Scrape the tables from the webpage

**To avoid in list we have done [0]**

In [112]:
table = webpage.select("table.hockey-stats")[0]
table

<table class="hockey-stats">
<thead>
<tr>
<th class="season" data-sort="">S</th>
<th class="team" data-sort="team">Team</th>
<th class="league" data-sort="league">League</th>
<th class="regular gp" data-sort="gp">GP</th>
<th class="regular g" data-sort="g">G</th>
<th class="regular a" data-sort="a">A</th>
<th class="regular tp" data-sort="tp">TP</th>
<th class="regular pim" data-sort="pim">PIM</th>
<th class="regular pm" data-sort="pm">+/-</th>
<th class="separator"> </th>
<th class="postseason">POST</th>
<th class="postseason gp" data-sort="playoffs-gp">GP</th>
<th class="postseason g" data-sort="playoffs-g">G</th>
<th class="postseason a" data-sort="playoffs-a">A</th>
<th class="postseason tp" data-sort="playoffs-tp">TP</th>
<th class="postseason pim" data-sort="playoffs-pim">PIM</th>
<th class="postseason pm" data-sort="playoffs-pm">+/-</th>
</tr>
</thead>
<tbody>
<tr class="team-continent-NA">
<td class="season sorted">
                  2014-15
              </td>
<td class="team"

In [113]:
table = webpage.select("table" , attr = {"class" : "hockey-stats"})[0]
table

<table class="hockey-stats">
<thead>
<tr>
<th class="season" data-sort="">S</th>
<th class="team" data-sort="team">Team</th>
<th class="league" data-sort="league">League</th>
<th class="regular gp" data-sort="gp">GP</th>
<th class="regular g" data-sort="g">G</th>
<th class="regular a" data-sort="a">A</th>
<th class="regular tp" data-sort="tp">TP</th>
<th class="regular pim" data-sort="pim">PIM</th>
<th class="regular pm" data-sort="pm">+/-</th>
<th class="separator"> </th>
<th class="postseason">POST</th>
<th class="postseason gp" data-sort="playoffs-gp">GP</th>
<th class="postseason g" data-sort="playoffs-g">G</th>
<th class="postseason a" data-sort="playoffs-a">A</th>
<th class="postseason tp" data-sort="playoffs-tp">TP</th>
<th class="postseason pim" data-sort="playoffs-pim">PIM</th>
<th class="postseason pm" data-sort="playoffs-pm">+/-</th>
</tr>
</thead>
<tbody>
<tr class="team-continent-NA">
<td class="season sorted">
                  2014-15
              </td>
<td class="team"

Table headers **th** can be considered as column names

Rows of the table **tr** can be considered as rows for the dataframe

**strip()** is used in table rows to remove trailing whitespaces

The cells will be appended in list l created

In [114]:
columns = table.find("thead").find_all("th")
column_names = [c.string for c in columns]

table_rows = table.find("tbody").find_all("tr")
l = []
for tr in table_rows :
  td = tr.find_all("td")
  row = [str(tr.get_text()).strip() for tr in td]
  l.append(row)

df = pd.DataFrame(l , columns = column_names)

df.head()

,S,Team,League,GP,G,A,TP,PIM,+/-,,POST,GP,G,A,TP,PIM,+/-
0,2014-15,MIT (Mass. Inst. of Tech.),ACHA II,17,3,9,12,20,,|,,,,,,,
1,2015-16,MIT (Mass. Inst. of Tech.),ACHA II,9,1,1,2,2,,|,,,,,,,
2,2016-17,MIT (Mass. Inst. of Tech.),ACHA II,12,5,5,10,8,0,|,,,,,,,
3,2017-18,Did not play,,,,,,,,|,,,,,,,
4,2018-19,MIT (Mass. Inst. of Tech.),ACHA III,8,5,10,15,8,,|,,,,,,,


In [115]:
df['Team']

0    MIT (Mass. Inst. of Tech.)
1    MIT (Mass. Inst. of Tech.)
2    MIT (Mass. Inst. of Tech.)
3                  Did not play
4    MIT (Mass. Inst. of Tech.)
Name: Team, dtype: object

In [116]:
df.loc[df['Team'] != "Did not play"]

,S,Team,League,GP,G,A,TP,PIM,+/-,,POST,GP,G,A,TP,PIM,+/-
0,2014-15,MIT (Mass. Inst. of Tech.),ACHA II,17,3,9,12,20,,|,,,,,,,
1,2015-16,MIT (Mass. Inst. of Tech.),ACHA II,9,1,1,2,2,,|,,,,,,,
2,2016-17,MIT (Mass. Inst. of Tech.),ACHA II,12,5,5,10,8,0,|,,,,,,,
4,2018-19,MIT (Mass. Inst. of Tech.),ACHA III,8,5,10,15,8,,|,,,,,,,


## iii. Grab all fun facts that have the word "is"

In [117]:
facts = webpage.select("ul.fun-facts li")
facts_with_is = [fact.find(string = re.compile("is")) for fact in facts]

#To remove facts_with_is which is false
facts_with_is = [fact.find_parent().get_text() for fact in facts_with_is if fact]
facts_with_is

['Middle name is Ronald',
 'Dunkin Donuts coffee is better than Starbucks',
 "A favorite book series of mine is Ender's Game",
 'Current video game of choice is Rocket League',
 "The band that I've seen the most times live is the Zac Brown Band"]

In [118]:
fun_facts = webpage.find_all("ul" , attrs = {"class" : "fun-facts"})[0]
facts = fun_facts.find_all("li")
facts_with_is = [fact.find(string = re.compile("is")) for fact in facts]

#To remove facts_with_is which is false
facts_with_is = [fact.find_parent().get_text() for fact in facts_with_is if fact]
facts_with_is

['Middle name is Ronald',
 'Dunkin Donuts coffee is better than Starbucks',
 "A favorite book series of mine is Ender's Game",
 'Current video game of choice is Rocket League',
 "The band that I've seen the most times live is the Zac Brown Band"]

## iv. Download an image

Note lake_como.jpg was written as it was a picture of lake como

In [119]:
images = webpage.select("div.row div.column img")
image_url = images[0]["src"]
full_url = url + image_url

img_data = requests.get(full_url).content 
with open('lake_como.jpg' , 'wb') as handler :
  handler.write(img_data)


## v. MYSTERY CHALLENGE

In [120]:
files = webpage.select("div.block a")
files

[<a href="challenge/file_1.html">File 1</a>,
 <a href="challenge/file_2.html">File 2</a>,
 <a href="challenge/file_3.html">File 3</a>,
 <a href="challenge/file_4.html">File 4</a>,
 <a href="challenge/file_5.html">File 5</a>,
 <a href="challenge/file_6.html">File 6</a>,
 <a href="challenge/file_7.html">File 7</a>,
 <a href="challenge/file_8.html">File 8</a>,
 <a href="challenge/file_9.html">File 9</a>,
 <a href="challenge/file_10.html">File 10</a>]

In [121]:
relative_files = [f['href'] for f in files]
relative_files

['challenge/file_1.html',
 'challenge/file_2.html',
 'challenge/file_3.html',
 'challenge/file_4.html',
 'challenge/file_5.html',
 'challenge/file_6.html',
 'challenge/file_7.html',
 'challenge/file_8.html',
 'challenge/file_9.html',
 'challenge/file_10.html']

In [123]:
url = "https://keithgalli.github.io/web-scraping/"
for f in relative_files :
  full_url = url + f
  page = requests.get(full_url)
  bs_page = bs(page.content)
  secret_word_element = bs_page.find("p" , attrs = {"id" : "secret-word"})
  secret_word = secret_word_element.string
  print(secret_word)

Make
sure
to
smash
that
like
button
and
subscribe
!!!
